# Pre-Process the data
Throw data of each gene to 

In [60]:
import GEOparse
import re
import os

## Extract Gene Info from GEO DataSets

In [1]:
gse = GEOparse.get_GEO(geo="GSE2770", destdir="./")

02-Aug-2019 15:17:13 DEBUG utils - Directory ./ already exists. Skipping.
02-Aug-2019 15:17:13 INFO GEOparse - File already exist: using local version.
02-Aug-2019 15:17:13 INFO GEOparse - Parsing ./GSE2770_family.soft.gz: 
02-Aug-2019 15:17:13 DEBUG GEOparse - DATABASE: GeoMiame
02-Aug-2019 15:17:13 DEBUG GEOparse - SERIES: GSE2770
02-Aug-2019 15:17:13 DEBUG GEOparse - PLATFORM: GPL96
02-Aug-2019 15:17:15 DEBUG GEOparse - PLATFORM: GPL97
02-Aug-2019 15:17:16 DEBUG GEOparse - PLATFORM: GPL8300
02-Aug-2019 15:17:16 DEBUG GEOparse - SAMPLE: GSM60348
02-Aug-2019 15:17:17 DEBUG GEOparse - SAMPLE: GSM60349
02-Aug-2019 15:17:17 DEBUG GEOparse - SAMPLE: GSM60350
02-Aug-2019 15:17:17 DEBUG GEOparse - SAMPLE: GSM60351
02-Aug-2019 15:17:17 DEBUG GEOparse - SAMPLE: GSM60352
02-Aug-2019 15:17:17 DEBUG GEOparse - SAMPLE: GSM60353
02-Aug-2019 15:17:17 DEBUG GEOparse - SAMPLE: GSM60354
02-Aug-2019 15:17:17 DEBUG GEOparse - SAMPLE: GSM60355
02-Aug-2019 15:17:17 DEBUG GEOparse - SAMPLE: GSM60356
02-Aug

## Extract Platform Information

In [36]:
keys = []
values = []
for gpl in gse.gpls:
    keys.append(gse.gpls[gpl].name)
    r1 = re.findall(r"\[.*?\]",gse.gpls[gpl].metadata['title'][0])
    values.append(r1[0][4:-1])
    #values.append(gse.gpls[gpl].metadata['title'][0])
    print(gse.gpls[gpl].name)
    print(gse.gpls[gpl].metadata['title'][0])
    
celformat = dict(zip(keys, values))

GPL96
[HG-U133A] Affymetrix Human Genome U133A Array
GPL97
[HG-U133B] Affymetrix Human Genome U133B Array
GPL8300
[HG_U95Av2] Affymetrix Human Genome U95 Version 2 Array


## Classify the Samples by Platform

In [59]:
keys = []
values = []
for gsm in gse.gsms:
    #print(gse.gsms[gsm].name)
    keys.append(gse.gsms[gsm].name)
    for key,val in celformat.items():
        r1 = re.findall(r"{}".format(val),gse.gsms[gsm].columns.loc['ID_REF','description'])
        if not r1:
            pass
        else:
            values.append(key)
    #r1 = re.findall(r"\(.*?\)",gse.gsms[gsm].columns.loc['ID_REF','description'])
    #values.append(r1[0][1:-1])
    #values.append(gse.gsms[gsm].columns.loc['ID_REF','description'])

gsm_platform = dict(zip(keys, values))
gsm_platform

{'GSM60348': 'GPL8300',
 'GSM60349': 'GPL8300',
 'GSM60350': 'GPL8300',
 'GSM60351': 'GPL8300',
 'GSM60352': 'GPL8300',
 'GSM60353': 'GPL8300',
 'GSM60354': 'GPL8300',
 'GSM60355': 'GPL8300',
 'GSM60356': 'GPL8300',
 'GSM60357': 'GPL8300',
 'GSM60358': 'GPL8300',
 'GSM60359': 'GPL8300',
 'GSM60360': 'GPL8300',
 'GSM60361': 'GPL8300',
 'GSM60362': 'GPL8300',
 'GSM60363': 'GPL8300',
 'GSM60364': 'GPL8300',
 'GSM60365': 'GPL8300',
 'GSM60366': 'GPL8300',
 'GSM60367': 'GPL8300',
 'GSM60368': 'GPL8300',
 'GSM60369': 'GPL8300',
 'GSM60370': 'GPL8300',
 'GSM60371': 'GPL8300',
 'GSM60372': 'GPL8300',
 'GSM60373': 'GPL8300',
 'GSM60374': 'GPL8300',
 'GSM60375': 'GPL8300',
 'GSM60376': 'GPL8300',
 'GSM60377': 'GPL8300',
 'GSM60378': 'GPL8300',
 'GSM60379': 'GPL8300',
 'GSM60380': 'GPL8300',
 'GSM60381': 'GPL8300',
 'GSM60699': 'GPL96',
 'GSM60700': 'GPL96',
 'GSM60701': 'GPL96',
 'GSM60702': 'GPL96',
 'GSM60703': 'GPL96',
 'GSM60704': 'GPL96',
 'GSM60705': 'GPL96',
 'GSM60706': 'GPL96',
 'GSM607

## Create Folders if not exist, Move files to corresponding folders

In [80]:
# Setup paths
currentdir = os.getcwd()
dirlist = currentdir.split('/')
projectdir = '/'.join(dirlist[0:-1])
datadir = os.path.join(projectdir,'data')
outputdir = os.path.join(projectdir,'output')
gene = 'GSE2770'
genedir = os.path.join(datadir,gene + '_RAW')

In [81]:
# create a folder for each platform
for key in celformat.keys():
    platformdir = os.path.join(genedir,key)
    if not os.path.exists(platformdir):
        os.makedirs(platformdir)
        print('Path created: {}'.format(platformdir))
    else:
        print('Path already exist: {}'.format(platformdir))


Path created: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL96
Path created: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97
Path created: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL8300


In [91]:
# Move Corresponding Cel files to Folders
#onlyfiles = [f for f in os.listdir(genedir) if os.path.isfile(os.path.join(genedir, f))]
onlyfiles = [f for f in os.listdir(genedir) if f.endswith('.gz')]

for file in onlyfiles:
    filelist = file.split('.')
    prefix = filelist[0]
    if prefix in gsm_platform:
        platform = gsm_platform[prefix]
        platformdir = os.path.join(genedir,platform)
        src_path = os.path.join(genedir, file)
        dst_path = os.path.join(platformdir, file)
        os.rename(src_path,dst_path)
        print('Move {} to {}'.format(src_path,dst_path))
        


Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60739.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60739.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60744.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60744.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60727.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL96/GSM60727.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60756.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60756.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60735.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60735.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GSM60748.CEL.gz to /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW/GPL97/GSM60748.CEL.gz
Move /Users/zhzhao/Dropbox/Helikar/pipel

In [90]:
# onlyfiles = [f for f in os.listdir(genedir) if os.path.isfile(os.path.join(genedir, f))]
# onlyfiles
# os.listdir(genedir)
onlyfiles = [f for f in os.listdir(genedir) if f.endswith('.gz')]
onlyfiles

['GSM60739.CEL.gz',
 'GSM60744.CEL.gz',
 'GSM60727.CEL.gz',
 'GSM60756.CEL.gz',
 'GSM60735.CEL.gz',
 'GSM60748.CEL.gz',
 'GSM60760.CEL.gz',
 'GSM60703.CEL.gz',
 'GSM60711.CEL.gz',
 'GSM60351.cel.gz',
 'GSM60379.CEL.gz',
 'GSM60367.CEL.gz',
 'GSM60375.CEL.gz',
 'GSM60353.cel.gz',
 'GSM60377.CEL.gz',
 'GSM60369.CEL.gz',
 'GSM60365.CEL.gz',
 'GSM60737.CEL.gz',
 'GSM60729.CEL.gz',
 'GSM60754.CEL.gz',
 'GSM60725.CEL.gz',
 'GSM60758.CEL.gz',
 'GSM60746.CEL.gz',
 'GSM60713.CEL.gz',
 'GSM60701.CEL.gz',
 'GSM60762.CEL.gz',
 'GSM60361.CEL.gz',
 'GSM60699.CEL.gz',
 'GSM60373.CEL.gz',
 'GSM60357.CEL.gz',
 'GSM60349.CEL.gz',
 'GSM60705.CEL.gz',
 'GSM60766.CEL.gz',
 'GSM60380.CEL.gz',
 'GSM60717.CEL.gz',
 'GSM60709.CEL.gz',
 'GSM60721.CEL.gz',
 'GSM60742.CEL.gz',
 'GSM60733.CEL.gz',
 'GSM60750.CEL.gz',
 'GSM60715.CEL.gz',
 'GSM60764.CEL.gz',
 'GSM60719.CEL.gz',
 'GSM60707.CEL.gz',
 'GSM60752.CEL.gz',
 'GSM60731.CEL.gz',
 'GSM60740.CEL.gz',
 'GSM60723.CEL.gz',
 'GSM60371.CEL.gz',
 'GSM60363.CEL.gz',


In [55]:
# dir(gse.gsms[gsm])
# print(gse.gsms[gsm].metadata['title'])
print(gse.gsms[gsm].columns.loc['ID_REF','description'])
r1 = re.findall(r"U133B",gse.gsms[gsm].columns.loc['ID_REF','description'])
r1

Affymetrix U95Av2 probe ID


[]

In [68]:
currentdir = os.getcwd()
dirlist = currentdir.split('/')
projectdir = '/'.join(dirlist[0:-1])

In [71]:
gene = 'GSE2770'
genedir = os.path.join(projectdir,gene + '_RAW')
genedir

'/Users/zhzhao/Dropbox/Helikar/pipelines/GSE2770_RAW'

In [37]:
celformat

{'GPL96': 'U133A', 'GPL97': 'U133B', 'GPL8300': 'U95Av2'}

In [30]:
r1 = re.findall(r"\[.*?\]",celformat['GPL96'])

In [35]:
r1[0][4:-1]

'U133A'